Suppose we have a `Polygon` class that has a vertices property that needs to be defined as a sequence of `Point2D` instances. So here, not only do we want the `vertices` attribute of our `Polygon` to be an iterable of some kind, we also want the elements to all be instances of the `Point2D` class. In turn we'll also want to make sure that coordinates for `Point2D` are non-negative integer values (as might be expected in computer screen coordinates):

In [ ]:
class Int:
    def __init__(self, min_value=None, max_value=None):
        self.min_value = min_value
        self.max_value = max_value
        
    def __set_name__(self, owner_class, name):
        self.name = name
        
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise ValueError(f'{self.name} must be an int.')
        if self.min_value is not None and value < self.min_value:
            raise ValueError(f'{self.name} must be at least {self.min_value}')
        if self.max_value is not None and value > self.max_value:
            raise ValueError(f'{self.name} cannot exceed {self.max_value}')
        instance.__dict__[self.name] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.name, None)

In [ ]:
class Point2D:
    x = Int(min_value=0, max_value=800)
    y = Int(min_value=0, max_value=400)
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Point2D(x={self.x}, y={self.y})'
    
    def __str__(self):
        return f'({self.x}, {self.y})'
    
    # Implementing equality comparison and __hash__ to make Point2D hashable
    def __eq__(self, other):
        return isinstance(other, Point2D) and self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash((self.x, self.y))


In [ ]:
# Point2D mutable sequence validator
import collections.abc

class Point2DSequence:
    def __init__(self, min_length=None, max_length=None):
        self.min_length = min_length
        self.max_length = max_length

    def __set_name__(self, owner_class, name):
        self.name = name

    def __set__(self, instance, value: collections.abc.Sequence):
        # the value should be a sequence type
        if not isinstance(value, collections.abc.Sequence):
            raise ValueError(f'{self.name} must be a sequence')
        # the sequence lenght should be between min_length and max_length
        if self.min_length is not None and len(value) < self.min_length:
            raise ValueError(f'{self.name} must contain at least {self.min_length} elements')
        if self.max_length is not None and len(value) > self.max_length:
            raise ValueError(f'{self.name} cannot contain more than {self.max_length} elements')
        # Every item in the sequence should be a Point2D instance
        for index, item in enumerate(value):
            if not isinstance(item, Point2D):
                raise ValueError(f'Item at index {index} is not a Point2D instance.')
        instance.__dict__[self.name] = value

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            if self.name not in instance.__dict__:
                # current point list has not been defined,
                # so let's create an empty list
                instance.__dict__[self.name] = []
            return instance.__dict__.get(self.name)
        
    def append(self, instance, value):
        if not isinstance(value, Point2D):
            raise ValueError('You can only append Point2D instances to the list.')
        if self.max_length is not None and len(instance.__dict__[self.name]) >= self.max_length:
            raise ValueError(f'{self.name} cannot contain more than {self.max_length} elements')
        
        instance.__dict__[self.name].append(value)



In [ ]:
class Polygon:
    vertices = Point2DSequence(min_length=3)
    
    def __init__(self, *vertices):
        self.vertices = vertices
        
    def append(self, pt):
        self.vertices.append(self, pt)

    # sequence protocol
    def __len__(self):
        return len(self.vertices)
    
    def __getitem__(self, s):
        return self.vertices[s]
    
    # Inplace addition
    def __iadd__(self, pt):
        self.append(pt)
        return self
    
    # containment check
    def __contains__(self, pt):
        return pt in self.vertices

In [ ]:
class Triangle(Polygon):
    vertices = Point2DSequence(min_length=3, max_length=3)

In [ ]:
class Square(Polygon):
    vertices = Point2DSequence(min_length=4, max_length=4)